<a href="https://colab.research.google.com/github/SzuHannah/ML-with-LCI/blob/main/ML_with_LCI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import log10
from numpy import arange, std, mean
from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.linear_model import Lasso, ElasticNet, MultiTaskElasticNet
from sklearn.preprocessing import MinMaxScaler

### Data exploration

In [ ]:
X = pd.read_csv("X_data.csv")

In [ ]:
X

,CNG pump-to-tank leakage,CNG tank mass intercept,CNG tank mass slope,CO2 per kg fuel,H2 tank mass per energy,LHV fuel MJ per kg,aerodynamic drag coefficient,auxilliary power base demand,average passenger mass,average passengers,battery DoD,battery cell energy density,"battery cell energy density, LFP","battery cell energy density, NCA","battery cell energy density, NMC","battery cell mass share, LFP","battery cell mass share, NCA","battery cell mass share, NMC",battery cell power density,battery cell production energy,battery cell production energy electricity share,battery charge efficiency,battery discharge efficiency,battery lifetime kilometers,battery onboard charging infrastructure cost,cargo mass,charger mass,combustion exhaust treatment cost,combustion fixed mass,combustion mass per power,combustion power share,combustion powertrain cost per kW,converter mass,cooling energy consumption,cooling thermal demand,drivetrain efficiency,electric fixed mass,electric mass per power,electric powertrain cost per kW,emission factor,energy battery cost per kWh,energy battery mass,energy cost per kWh,engine efficiency,frontal area,fuel cell ancillary BoP mass per power,fuel cell cost per kW,fuel cell essential BoP mass per power,fuel cell lifetime hours,fuel cell own consumption,fuel cell power area density,fuel cell power share,fuel cell stack efficiency,fuel mass,fuel tank cost per kg,fuel tank mass per energy,glider base mass,glider cost intercept,glider cost slope,glider lightweighting cost per kg,heat pump cost,heating energy consumption,heating thermal demand,interest rate,inverter mass,kilometers per year,lifetime kilometers,lightweighting,maintenance cost per glider cost,markup factor,power battery cost per kW,power distribution unit mass,power to mass ratio,powertrain fixed mass,powertrain mass per power,rolling resistance coefficient,country,size,powertrain,year
0,0.0,0.0,0.0,3.18334,0.0,42.125236,0.299565,96.033956,85.632944,1.595889,0.0,0.0,0.052759,0.0,0.0,0.470322,0.460428,0.483038,2.067906,29.880607,0.554314,0.842460,0.0,246264.647003,0.0,141.335959,0.0,553.354287,60.143680,0.673356,1.000000,29.917635,0.0,1.092890,342.194199,0.711573,14.763662,0.437591,35.666269,1.215441,0.0,0.0,0.169819,0.195092,2.353024,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.074579,0.567422,0.073523,1527.166574,-1551.215110,14.836389,5.320266,0.0,0.0,349.077960,0.049057,0.000000,9663.762898,237819.327575,0.169420,0.031098,1.530583,85.864738,0.000000,95.839317,58.576313,0.550380,0.011126,CH,Large,ICEV-p,2000
1,0.0,0.0,0.0,3.18334,0.0,42.858906,0.291894,106.054747,70.125920,2.855516,0.0,0.0,0.050716,0.0,0.0,0.470397,0.474200,0.546618,1.930198,33.284764,0.743941,0.842698,0.0,199360.954472,0.0,78.216255,0.0,567.044076,57.812170,0.745477,1.000000,46.370387,0.0,1.009113,313.477869,0.671267,13.751066,0.313390,47.460535,1.058174,0.0,0.0,0.152189,0.200872,2.367549,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.789660,0.486714,0.073397,1628.974020,-968.477517,16.590053,3.590378,0.0,0.0,410.350584,0.056563,0.000000,11573.634271,151515.545241,0.176757,0.038506,1.725005,58.471697,0.000000,72.372148,48.272417,0.523276,0.008982,CH,Large,ICEV-p,2000
2,0.0,0.0,0.0,3.18334,0.0,42.269949,0.276677,97.657349,82.457675,3.289820,0.0,0.0,0.049306,0.0,0.0,0.469389,0.505101,0.563510,1.693372,31.815491,0.277921,0.844560,0.0,267458.911296,0.0,75.842041,0.0,643.274162,63.630420,0.675908,1.000000,35.189729,0.0,1.024363,358.619951,0.741158,15.619997,0.460584,50.365051,0.901185,0.0,0.0,0.168471,0.194263,2.399606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.737642,0.688575,0.072591,1482.630640,-1106.554536,13.897000,4.394066,0.0,0.0,350.604732,0.039345,0.000000,11368.448281,143304.084962,0.044539,0.027794,1.479433,40.987647,0.000000,69.737359,49.411367,0.535621,0.010556,CH,Large,ICEV-p,2000
3,0.0,0.0,0.0,3.18334,0.0,42.304851,0.285477,116.001304,75.214997,2.805289,0.0,0.0,0.050110,0.0,0.0,0.524098,0.437234,0.502622,2.217103,36.642001,0.634583,0.864344,0.0,251062.570299,0.0,193.844663,0.0,552.284360,64.61541

In [ ]:
numericcols = []
numeric_X = X.apply(lambda x: numericcols.append(x.name) if np.issubdtype(x.dtype, np.number) else x)
len(numericcols)

77

In [ ]:
X.describe()

,CNG pump-to-tank leakage,CNG tank mass intercept,CNG tank mass slope,CO2 per kg fuel,H2 tank mass per energy,LHV fuel MJ per kg,aerodynamic drag coefficient,auxilliary power base demand,average passenger mass,average passengers,battery DoD,battery cell energy density,"battery cell energy density, LFP","battery cell energy density, NCA","battery cell energy density, NMC","battery cell mass share, LFP","battery cell mass share, NCA","battery cell mass share, NMC",battery cell power density,battery cell production energy,battery cell production energy electricity share,battery charge efficiency,battery discharge efficiency,battery lifetime kilometers,battery onboard charging infrastructure cost,cargo mass,charger mass,combustion exhaust treatment cost,combustion fixed mass,combustion mass per power,combustion power share,combustion powertrain cost per kW,converter mass,cooling energy consumption,cooling thermal demand,drivetrain efficiency,electric fixed mass,electric mass per power,electric powertrain cost per kW,emission factor,energy battery cost per kWh,energy battery mass,energy cost per kWh,engine efficiency,frontal area,fuel cell ancillary BoP mass per power,fuel cell cost per kW,fuel cell essential BoP mass per power,fuel cell lifetime hours,fuel cell own consumption,fuel cell power area density,fuel cell power share,fuel cell stack efficiency,fuel mass,fuel tank cost per kg,fuel tank mass per energy,glider base mass,glider cost intercept,glider cost slope,glider lightweighting cost per kg,heat pump cost,heating energy consumption,heating thermal demand,interest rate,inverter mass,kilometers per year,lifetime kilometers,lightweighting,maintenance cost per glider cost,markup factor,power battery cost per kW,power distribution unit mass,power to mass ratio,powertrain fixed mass,powertrain mass per power,rolling resistance coefficient,year
count,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000,15876.000000
mean,0.003900,2.519396,0.019701,1.829119,0.056699,38.425008,0.312600,76.813736,73.754576,2.208295,0.166564,0.064595,0.046959,0.067545,0.064595,0.678668,0.523158,0.511759,1.240361,17.044816,0.508712,0.843139,0.181804,175094.121696,56.413952,107.886899,1.872679,460.032821,37.468961,0.491486,0.537772,44.320692,2.151034,0.840697,270.367470,0.747085,17.057704,0.514523,31.896715,1.030497,137.486899,67.977884,0.140923,0.450773,2.455607,0.035587,24.993317,0.092784,303.889225,0.112522,90.707218,0.073633,0.050825,28.271499,111.982924,0.044175,1147.137885,-1539.357027,12.744209,4.988573,35.671458,0.193202,292.588370,0.051394,5.905746,10430.892347,183414.808993,0.101704,0.027311,1.383273,55.048704,2.643479,68.464369,49.585700,0.503076,0.009255,2017.500000
std,0.013809,7.644117,0.059556,1.386362,0.171965,32.844404,0.030548,20.258748,5.803650,0.606703,0.285414,0.111559,0.056629,0.115912,0.111559,0.207710,0.165904,0.167058,1.065212,10.088086,0.176263,0.014854,0.311798,87801.679627,82.002036,61.995322,2.670734,323.071566,27.451043,0.332685,0.409373,35.808531,2.431605,0.320601,113.574852,0.230732,3.912715,0.081065,15.

In [ ]:
# plot distribution of features
# plt.figure(figsize=(24,200))
# try:
#     for i, col in enumerate(X.columns.to_list()):
#         plt.subplot(20, 4, i + 1)
#         plt.hist(X[col], label=col,color='blue')
#         plt.legend()
#         plt.title(col)
#         plt.tight_layout()
# except Exception as e:
#     print(col,e)

In [ ]:
X['size'],sizetype=pd.factorize(X['size'])
X['country'],countrytype = pd.factorize(X['country'])
X['powertrain'],powertrain = pd.factorize(X['powertrain'])

In [ ]:
#normalize numeric cols
# for i in numericcols:
#   norm = MinMaxScaler().fit(X[[i]])
#   X[i] = norm.transform(X[[i]])

# normalize all cols
# norm = MinMaxScaler().fit(X)
# X = norm.transform(X)

# mean absolute error (MAE) was lower for non-normalization data, and more predicted coeffs were nonzero (with normalized data, only two coeffs were nonzero)

In [ ]:
X

,CNG pump-to-tank leakage,CNG tank mass intercept,CNG tank mass slope,CO2 per kg fuel,H2 tank mass per energy,LHV fuel MJ per kg,aerodynamic drag coefficient,auxilliary power base demand,average passenger mass,average passengers,battery DoD,battery cell energy density,"battery cell energy density, LFP","battery cell energy density, NCA","battery cell energy density, NMC","battery cell mass share, LFP","battery cell mass share, NCA","battery cell mass share, NMC",battery cell power density,battery cell production energy,battery cell production energy electricity share,battery charge efficiency,battery discharge efficiency,battery lifetime kilometers,battery onboard charging infrastructure cost,cargo mass,charger mass,combustion exhaust treatment cost,combustion fixed mass,combustion mass per power,combustion power share,combustion powertrain cost per kW,converter mass,cooling energy consumption,cooling thermal demand,drivetrain efficiency,electric fixed mass,electric mass per power,electric powertrain cost per kW,emission factor,energy battery cost per kWh,energy battery mass,energy cost per kWh,engine efficiency,frontal area,fuel cell ancillary BoP mass per power,fuel cell cost per kW,fuel cell essential BoP mass per power,fuel cell lifetime hours,fuel cell own consumption,fuel cell power area density,fuel cell power share,fuel cell stack efficiency,fuel mass,fuel tank cost per kg,fuel tank mass per energy,glider base mass,glider cost intercept,glider cost slope,glider lightweighting cost per kg,heat pump cost,heating energy consumption,heating thermal demand,interest rate,inverter mass,kilometers per year,lifetime kilometers,lightweighting,maintenance cost per glider cost,markup factor,power battery cost per kW,power distribution unit mass,power to mass ratio,powertrain fixed mass,powertrain mass per power,rolling resistance coefficient,country,size,powertrain,year
0,0.0,0.0,0.0,3.18334,0.0,42.125236,0.299565,96.033956,85.632944,1.595889,0.0,0.0,0.052759,0.0,0.0,0.470322,0.460428,0.483038,2.067906,29.880607,0.554314,0.842460,0.0,246264.647003,0.0,141.335959,0.0,553.354287,60.143680,0.673356,1.000000,29.917635,0.0,1.092890,342.194199,0.711573,14.763662,0.437591,35.666269,1.215441,0.0,0.0,0.169819,0.195092,2.353024,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.074579,0.567422,0.073523,1527.166574,-1551.215110,14.836389,5.320266,0.0,0.0,349.077960,0.049057,0.000000,9663.762898,237819.327575,0.169420,0.031098,1.530583,85.864738,0.000000,95.839317,58.576313,0.550380,0.011126,0,0,0,2000
1,0.0,0.0,0.0,3.18334,0.0,42.858906,0.291894,106.054747,70.125920,2.855516,0.0,0.0,0.050716,0.0,0.0,0.470397,0.474200,0.546618,1.930198,33.284764,0.743941,0.842698,0.0,199360.954472,0.0,78.216255,0.0,567.044076,57.812170,0.745477,1.000000,46.370387,0.0,1.009113,313.477869,0.671267,13.751066,0.313390,47.460535,1.058174,0.0,0.0,0.152189,0.200872,2.367549,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.789660,0.486714,0.073397,1628.974020,-968.477517,16.590053,3.590378,0.0,0.0,410.350584,0.056563,0.000000,11573.634271,151515.545241,0.176757,0.038506,1.725005,58.471697,0.000000,72.372148,48.272417,0.523276,0.008982,0,0,0,2000
2,0.0,0.0,0.0,3.18334,0.0,42.269949,0.276677,97.657349,82.457675,3.289820,0.0,0.0,0.049306,0.0,0.0,0.469389,0.505101,0.563510,1.693372,31.815491,0.277921,0.844560,0.0,267458.911296,0.0,75.842041,0.0,643.274162,63.630420,0.675908,1.000000,35.189729,0.0,1.024363,358.619951,0.741158,15.619997,0.460584,50.365051,0.901185,0.0,0.0,0.168471,0.194263,2.399606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.737642,0.688575,0.072591,1482.630640,-1106.554536,13.897000,4.394066,0.0,0.0,350.604732,0.039345,0.000000,11368.448281,143304.084962,0.044539,0.027794,1.479433,40.987647,0.000000,69.737359,49.411367,0.535621,0.010556,0,0,0,2000
3,0.0,0.0,0.0,3.18334,0.0,42.304851,0.285477,116.001304,75.214997,2.805289,0.0,0.0,0.050110,0.0,0.0,0.524098,0.437234,0.502622,2.217103,36.642001,0.634583,0.864344,0.0,251062.570299,0.0,193.844663,0.0,552.284360,64.615415,0.627492,1.000000,46.826620,

In [ ]:
 # split train test
 # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### train/test with existing library
#### Prediction for one response
- predict for each respoonse separately, e.g., climate change; fossil depletion; metal depletion; human_toxicity,... 
- package:   
  - Lasso from sklearn  
  - Elastic net (L1/L2) from sklearn 

In [ ]:
y_cc = pd.read_csv("Y_data.csv", usecols = ["climate change"])
y_fd = pd.read_csv("Y_data.csv", usecols = ["fossil depletion"])
y_md = pd.read_csv("Y_data.csv", usecols = ["metal depletion"])
y_ht = pd.read_csv("Y_data.csv", usecols = ["human toxicity"])

##### Lasso from sklearn

In [ ]:
def MLwithSKLasso(X, y):
  '''return r2, best lambda, coefficients of the features, features that are nonzero'''
  # ref: https://machinelearningmastery.com/lasso-regression-with-python/
  model = Lasso()
  # model evaluation method - cross-validation
  cv = RepeatedKFold(n_splits = 5, n_repeats = 3, random_state = 1)
  # define grid, i.e. range and gap of lambda to test
  grid = dict()
  grid['alpha'] = arange(0.05,1, 0.01) # does not converge when alpha = 0, so start from 0.05
  # define how to search for optimal lamda
  search = GridSearchCV(model, grid, scoring='r2', cv=cv, n_jobs=-1)
  results = search.fit(X,y)
  # summarize result
  r2 = results.best_score_
  optimal_lambda = results.best_params_
  coefficients = results.best_estimator_.coef_
  nonzerofeature_index = np.argwhere(coefficients != 0).flatten()
  critical_features= X.columns[nonzerofeature_index]

  return r2, optimal_lambda,coefficients, critical_features.tolist()

In [ ]:
# climate change prediction result
r2_cc, best_lambda_cc, coef_cc, critical_feat_cc = MLwithSKLasso(X,y_cc)
print('R2: %.3f' % r2_cc)
print('Config: %s' % best_lambda_cc)
critical_feat_cc

R2: 0.646
Config: {'alpha': 0.05}


['CNG tank mass intercept',
 'auxilliary power base demand',
 'average passenger mass',
 'battery cell production energy',
 'battery lifetime kilometers',
 'battery onboard charging infrastructure cost',
 'combustion exhaust treatment cost',
 'combustion fixed mass',
 'combustion powertrain cost per kW',
 'cooling thermal demand',
 'electric fixed mass',
 'electric powertrain cost per kW',
 'energy battery cost per kWh',
 'energy battery mass',
 'fuel cell power area density',
 'fuel mass',
 'fuel tank cost per kg',
 'glider base mass',
 'glider cost intercept',
 'heat pump cost',
 'inverter mass',
 'kilometers per year',
 'lifetime kilometers',
 'power to mass ratio',
 'year']

In [ ]:
# fossil depletion prediction result
r2_fd, best_lambda_fd, coef_fd, critical_feat_fd = MLwithSKLasso(X, y_fd)
print('R2: %.3f' % r2_fd)
print('Config: %s' % best_lambda_fd)
critical_feat_fd

R2: 0.545
Config: {'alpha': 0.05}


['CNG tank mass intercept',
 'auxilliary power base demand',
 'battery cell production energy',
 'battery lifetime kilometers',
 'battery onboard charging infrastructure cost',
 'cargo mass',
 'combustion exhaust treatment cost',
 'cooling thermal demand',
 'electric powertrain cost per kW',
 'energy battery cost per kWh',
 'energy battery mass',
 'fuel cell lifetime hours',
 'fuel tank cost per kg',
 'glider base mass',
 'glider cost intercept',
 'heat pump cost',
 'kilometers per year',
 'lifetime kilometers',
 'year']

In [ ]:
# metal depletion prediction result
r2_md, best_lambda_md, coef_md, critical_feat_md = MLwithSKLasso(X, y_md)
print('R2: %.3f' % r2_md)
print('Config: %s' % best_lambda_md)
critical_feat_md

R2: 0.430
Config: {'alpha': 0.05}


['battery lifetime kilometers',
 'battery onboard charging infrastructure cost',
 'cargo mass',
 'combustion exhaust treatment cost',
 'energy battery cost per kWh',
 'energy battery mass',
 'fuel cell lifetime hours',
 'fuel tank cost per kg',
 'glider base mass',
 'glider cost intercept',
 'heat pump cost',
 'heating thermal demand',
 'kilometers per year',
 'lifetime kilometers',
 'year']

In [ ]:
r2_ht, best_lambda_ht, coef_ht, critical_feat_ht = MLwithSKLasso(X, y_ht)
print('R2: %.3f' % r2_ht)
print('Config: %s' % best_lambda_ht)
critical_feat_ht

R2: 0.472
Config: {'alpha': 0.05}


['auxilliary power base demand',
 'battery cell production energy',
 'battery lifetime kilometers',
 'battery onboard charging infrastructure cost',
 'cargo mass',
 'combustion exhaust treatment cost',
 'combustion fixed mass',
 'electric powertrain cost per kW',
 'energy battery cost per kWh',
 'energy battery mass',
 'fuel cell lifetime hours',
 'fuel cell power area density',
 'fuel tank cost per kg',
 'glider base mass',
 'glider cost intercept',
 'heat pump cost',
 'heating thermal demand',
 'kilometers per year',
 'lifetime kilometers',
 'power to mass ratio',
 'year']

In [ ]:
# plot a directed graph to represent critical feature and response
# input node
critical_feats = set(critical_feat_cc + critical_feat_fd+critical_feat_ht+critical_feat_md)
def create_feat_response_df(feature, response):
  n_feat = len(feature)
  data = pd.DataFrame({'feature':feature, 'reponse':[response]*n_feat})
  return data

feat_response_cc = create_feat_response_df(critical_feat_cc,"climate change")
feat_response_fd = create_feat_response_df(critical_feat_fd,"fossil depletion")
feat_response_ht = create_feat_response_df(critical_feat_ht,"human toxicity")
feat_response_md = create_feat_response_df(critical_feat_md,"metal depletion")

In [ ]:
edges_df = pd.concat([feat_response_cc,feat_response_fd,feat_response_ht,feat_response_md])
edges_df.to_csv("feat_response_lasso_singleOutputTask.csv",index = False)

In [ ]:
#!pip install pyvis
# ref: https://pyvis.readthedocs.io/en/latest/index.html 
from pyvis import network as net
from IPython.core.display import display, HTML
import networkx as nx

g = net.Network("500px","800px",directed = True, notebook=True)
g.barnes_hut()
diG = nx.DiGraph()
for feat, resp in edges_df.values:
  diG.add_edge(feat,resp)

pos = nx.circular_layout(diG)
fig = plt.figure(1, figsize=(30, 15));

g.from_nx(diG)
g.show("feat_response_singleOutTask_lasso.html")
display(HTML('feat_response_singleOutTask_lasso.html')) #https://github.com/WestHealth/pyvis/issues/49


<Figure size 2160x1080 with 0 Axes>

##### Elastic Net

In [ ]:
def MLwithSKElastic(X,y):
  '''return r2, best lambda, coefficients of the features, features that are nonzero'''
  # ref: https://machinelearningmastery.com/elastic-net-regression-in-python/; 
  model = ElasticNet()
  # model evaluation method - cross-validation
  cv = RepeatedKFold(n_splits = 5, n_repeats = 3, random_state = 1)
  # define grid, i.e. range and gap of alpha/l1_ratio to test (if l1_ratio is decided, l2_ratio follows)
  grid = dict()
  grid['alpha'] = [0.05, 0.1, 1] #controls sum of both l1 and l2 penalties
  grid['l1_ratio'] = arange(0.05,0.5,0.05)
  # define how to search for optimal alpha/l1_ratio
  search = GridSearchCV(model, grid, scoring='r2', cv=cv, n_jobs=-1)
  results = search.fit(X,y)
  # summarize result
  r2 = results.best_score_
  optimal_params = results.best_params_ #optimal alpha and l1_ratio
  coefficients = results.best_estimator_.coef_
  nonzerofeature_index = np.argwhere(coefficients != 0).flatten()
  critical_features= X.columns[nonzerofeature_index]

  return r2, optimal_params,coefficients, critical_features.tolist()  


In [ ]:
# climate change prediction result
elasr2_cc, elasbest_param_cc, elascoef_cc, elascritical_feat_cc = MLwithSKElastic(X,y_cc)
print('R2: %.3f' % elasr2_cc)
print('Config: %s' % elasbest_param_cc)
elascritical_feat_cc

R2: 0.684
Config: {'alpha': 0.05, 'l1_ratio': 0.05}


['CNG tank mass intercept',
 'LHV fuel MJ per kg',
 'auxilliary power base demand',
 'average passenger mass',
 'average passengers',
 'battery cell power density',
 'battery cell production energy',
 'battery lifetime kilometers',
 'battery onboard charging infrastructure cost',
 'cargo mass',
 'combustion exhaust treatment cost',
 'combustion fixed mass',
 'combustion powertrain cost per kW',
 'cooling thermal demand',
 'electric fixed mass',
 'electric powertrain cost per kW',
 'energy battery cost per kWh',
 'energy battery mass',
 'fuel cell cost per kW',
 'fuel cell lifetime hours',
 'fuel cell power area density',
 'fuel mass',
 'fuel tank cost per kg',
 'glider base mass',
 'glider cost intercept',
 'glider cost slope',
 'heat pump cost',
 'heating thermal demand',
 'inverter mass',
 'kilometers per year',
 'lifetime kilometers',
 'power battery cost per kW',
 'power to mass ratio',
 'powertrain fixed mass',
 'country',
 'size',
 'year']

In [ ]:
# compare critical_feat_cc from Lasso and Elastic net
#pd.DataFrame({"Lasso_feature":critical_feat_cc, "Elastic_feature":elascritical_feat_cc})
lf_cc = pd.DataFrame({"Lasso_feature":critical_feat_cc})
ef_cc = pd.DataFrame({"Elastic_feature":elascritical_feat_cc})
lf_ef_cc = ef_cc.join(lf_cc)
lf_ef_cc
lf_ef_cc.to_csv("lasso_elas_fetcc_singleOutputTask.csv",index = False)

#### Prediction for multi-response  
- predicted respoonses altogether, e.g., climate change; fossil depletion; metal depletion; human_toxicity,... 
- packages:  
  - multi-task elastic net from sklearn (but this might not be appropriate, since it'll select the same subset of input features for the responses)  
  - Neural Network

In [ ]:
Y = pd.read_csv("Y_data.csv", usecols = ["climate change","fossil depletion","metal depletion", "human toxicity"])

##### MultiTask ElasticNet  
jointly select features for all responses

In [ ]:
def MLwithSKMultiElastic(X,Y):
  '''return r2, coefficients of the features, features that are nonzero'''
  model = MultiTaskElasticNet(alpha = 0.05, l1_ratio = 0.05)
  #cross-validation with grid search took too much time... for now, use alpha=0.05, l1_ratio = 0.05, the optimal result from previous monotask
  cv = RepeatedKFold(n_splits = 5, n_repeats = 3, random_state=1)
  scores = cross_val_score(model, X, Y, scoring = 'r2', cv = cv, n_jobs = -1)
  # summarize result
  r2 = 'R2: %.3f(%.3f)'%(mean(scores), std(scores))
  model.fit(X,Y)
  coefficients = model.coef_
  nonzerofeature_index = np.argwhere(coefficients[0] != 0).flatten()
  critical_features= X.columns[nonzerofeature_index]

  return r2, coefficients, critical_features.tolist()  

In [ ]:
multi_elasr2, multi_elascoef, multi_elascritical_feat = MLwithSKMultiElastic(X,Y)
len(multi_elascritical_feat), multi_elasr2

(39, 'R2: 0.655(0.011)')

In [ ]:
multi_elascritical_feat_df = pd.DataFrame({"feature": multi_elascritical_feat})
multi_elascritical_feat_df.to_csv("multitask_elasFeat_sameforall.csv", index = None)

In [ ]:
#test
model = MultiTaskElasticNet(alpha = 0.05, l1_ratio = 0.05)
cv = RepeatedKFold(n_splits = 5, n_repeats = 3, random_state=1)
scores = cross_val_score(model, X, Y, scoring = 'neg_mean_squared_error', cv = cv, n_jobs = -1) #taking really long.. maybe change coordinate descent to proximal descent
print('MSE: %.3f(%.3f)'%(mean(scores), std(scores)))
model.fit(X,Y)
coefficients = model.coef_
#coefficients
# check if same set of inputs were returned for all responses -> yes
# for c in coefficients:
#   nonzero_i = np.argwhere(c!=0).flatten()
#   critical_feats = X.columns[nonzero_i]
#   print(critical_feats)
#   print(len(critical_feats))

MSE: -0.002(0.000)


In [ ]:
nonzerofeature_index = np.argwhere(coefficients[0]!=0).flatten()
critical_feats = X.columns[nonzerofeature_index]
print(critical_feats)

Index(['CNG tank mass intercept', 'LHV fuel MJ per kg',
       'auxilliary power base demand', 'average passenger mass',
       'average passengers', 'battery cell power density',
       'battery cell production energy', 'battery lifetime kilometers',
       'battery onboard charging infrastructure cost', 'cargo mass',
       'charger mass', 'combustion exhaust treatment cost',
       'combustion fixed mass', 'combustion powertrain cost per kW',
       'cooling thermal demand', 'electric fixed mass',
       'electric powertrain cost per kW', 'energy battery cost per kWh',
       'energy battery mass', 'fuel cell cost per kW',
       'fuel cell lifetime hours', 'fuel cell power area density', 'fuel mass',
       'fuel tank cost per kg', 'glider base mass', 'glider cost intercept',
       'glider cost slope', 'heat pump cost', 'heating thermal demand',
       'inverter mass', 'kilometers per year', 'lifetime kilometers',
       'power battery cost per kW', 'power distribution unit mass',

##### Neural Network

In [ ]:
numericcols = list(X.select_dtypes('float64').columns)
numericcols.append("year")
catcols = list(X.select_dtypes('int64').columns)
catcols.remove("year")
len(numericcols), len(catcols)

(77, 3)

In [ ]:
# standardize data to make life easier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,OneHotEncoder

pipeline=ColumnTransformer(
    transformers = [
      ('num',StandardScaler(),numericcols), # standardized numeric features
      ('cat',OneHotEncoder(),catcols), #one-hot code categorical features
    ]
)

standX = pipeline.fit_transform(X)
standY = StandardScaler().fit_transform(Y)

In [ ]:
len(standX[1000])

102

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense
from keras.optimizers import gradient_descent_v2

In [ ]:
# split into train and test
standX_train, standX_test, standY_train, standY_test = train_test_split(standX, standY, test_size = 0.2, random_state = 42)
# define model, ref: https://machinelearningmastery.com/deep-learning-models-for-multi-output-regression/; https://keras.io/api/models/model_training_apis/
model = Sequential()
model.add(Dense(20, input_dim=102, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(4, activation='linear'))
opt = gradient_descent_v2.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mse'])
# fit model
history = model.fit(standX_train, standY_train, validation_data=(standX_test, standY_test), epochs=100, verbose=0)
# evaluate the model
_,train_mse = model.evaluate(standX_train, standY_train, verbose=0)
_,test_mse = model.evaluate(standX_test, standY_test, verbose=0)
print('Train: %.3f; Test: %.3f' % (train_mse, test_mse))

Train: 0.005; Test: 0.006


In [ ]:
# test customed multitask (not better for now)
def build_model():
  # model layers
  inp_layer = Input(shape=(102,))
  first_dense = Dense(units = '30', activation = 'relu')(inp_layer)
  second_dense = Dense(units = '20', activation = 'relu')(first_dense)

  Y1_out = Dense(units = '1', name = 'human_toxicity')(second_dense)
  Y2_out = Dense(units = '1', name = 'metal_depletion')(second_dense)
  Y3_out = Dense(units = '1', name = 'climate_change')(second_dense)
  Y4_out = Dense(units = '1', name = 'fossil_depletion')(second_dense)
  # construct the model with the input layer and a list of output layers
  model = Model(inputs = inp_layer, outputs = [Y1_out, Y2_out, Y3_out, Y4_out])

  return model

# use the model
model = build_model()
opt = gradient_descent_v2.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss='mean_squared_error', optimizer=opt, metrics={'human_toxicity': 'mse','metal_depletion': 'mse','climate_change': 'mse','fossil_depletion': 'mse'})
# fit model
history = model.fit(standX_train, standY_train, validation_data=(standX_test, standY_test), epochs=100, verbose=0)
# evaluate the model
_,_,_,_,_, trainY1_mse, trainY2_mse, trainY3_mse, trainY4_mse = model.evaluate(standX_train, standY_train, verbose=0)
_,_,_,_,_, testY1_mse, testY2_mse, testY3_mse, testY4_mse = model.evaluate(standX_test, standY_test, verbose=0)
print('Train: %.3f, %.3f, %.3f, %.3f,; Test: %.3f, %.3f, %.3f, %.3f' % (trainY1_mse, trainY2_mse, trainY3_mse, trainY4_mse,testY1_mse, testY2_mse, testY3_mse, testY4_mse))